### 使用 piecewise Rossler 模型模拟爆炸同步现象
ref: I. Leyva, et al. Explosive first-order transition to synchrony in networked chaotic oscillators, Phys. Rev. Lett. 108, 168702 (2012).

model: piecewise Rossler model

network: scale-free network

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange
import random
import networkx as nx
from scipy.sparse import coo_matrix
from joblib import Parallel, delayed

In [ ]:
# 演算法
@njit
def RK4(fun, x0, t, dt, *args):
    """
    使用 Runge-Kutta 方法计算一个时间步后系统的状态。
    args:
        fun: 微分方程
        x0: 上一个时间单位的状态变量
        t: 运行时间
        dt: 时间步长
    :return:
        x0 (numpy.ndarray): 下一个时间单位的状态变量
    """
    k1 = fun(x0, t, *args)
    k2 = fun(x0 + (dt / 2.) * k1, t + (dt / 2.), *args)
    k3 = fun(x0 + (dt / 2.) * k2, t + (dt / 2.), *args)
    k4 = fun(x0 + dt * k3, t + dt, *args)

    x0 += (dt / 6.) * (k1 + 2 * k2 + 2 * k3 + k4)

    return x0

# piecewise Rossler model
@njit
def piecewise_Rossler(vars, t, I, alpha, beta, R, mu, gamma):
    res = np.zeros_like(vars)
    # 状态变量
    x, y, z = vars

    g_x = 0 if x <= 3 else mu*(x - 3)

    dx_dt = gamma * (-alpha*(x - I) - z - beta*y)
    dy_dt = gamma * (x - (0.02 - 10/R)*y)
    dz_dt = gamma * (g_x - z)

    res[0] = dx_dt
    res[1] = dy_dt
    res[2] = dz_dt

    return res

In [ ]:
# 耦合函数
@njit
def coupling_func(vars, d, ):
    pass


In [ ]:
@njit
def net(N, ):
    # ============ 设置节点参数 ============ 
    alpha = 0.05
    beta = 0.5
    R = 100                 # chaos(R: 55-110)
    mu = 15
    gamma_base = 10000  

    x0 = 0.1*np.ones(N)
    y0 = 0.2*np.ones(N)
    z0 = 0.3*np.ones(N)
    vars_nodes = np.vstack((x0, y0, z0))        # 状态变量矩阵
    N_vars = 3          # 变量的数量
